# Analyzing Presidential Speeches using Artificial Neural Networks
When building NLP classifiers, the goal is usually to identify the class of new, unseen data. This project aims to take this one step further by not only identifying the correct speaker of presidential quotes, but to demonstrate how such an approach can provide to not just the differences, but similarities between presidents.

All data for this project can be found in the folder labeled *Corpus of Presidential Speeches,* which contains over 1,000 raw text files, each of which represents a single speech given by a president. Also included are the general campaign speeches for Donald Trump and Hillary Clinton, though the data does not contain speeches given by Trump since taking office. The original corpus was can be accessed from **[The Grammar Lab](http://www.thegrammarlab.com/?nor-portfolio=corpus-of-presidential-speeches-cops-and-a-clintontrump-corpus)**.

The term *speech* is broadly defined by The Grammar Lab, and may include other forms of public speaking, including debate transcripts. For purposes of this analysis, context of public speech is of little concern, so the corpora was accepted as is.

**IMPORTANT: In order for this notebook to run properly, you will first need to download the pre-trained GloVe vectors [here](http://nlp.stanford.edu/data/glove.6B.zip). The file glove.6B.100d.txt will need to be then saved to the same folder as this notebook file.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, string, re
import datetime
import warnings

warnings.filterwarnings('ignore')
np.random.seed(123)

## 1. Select Presidents for Comparison
Select any two presidents you want, as well as the minimum word count for each sample. All examples previously used 50 word samples, but you can choose any size you like. Larger samples may increase accuracy, but will also make it more difficult to understand how and why the network got it wrong. It will also result in fewer samples, meaning there may be greater levels of variance in the results. Maybe try a few different options?

In [ ]:
def get_presidents():
    # Create dictionary with each key representing an index number, and each value representing a president name
    # Names are taken from directory of folders, each of which contains the corpus of speeches for that individual
    
    i=1
    selection_index = {}
    for pres in sorted(list(os.listdir('Corpus of Presidential Speeches/')))[1:]:
        print(f'{i}:\t{pres.title()}')
        selection_index[i] = pres
        i += 1
    
    # Get input from user to choose presidents to compare, along with minimum vocabulary size for each sample
    selections = [input('\n\nEnter the number of the first president/candidate you would like to compare:\t')]
    selections.append(input('Enter the number of the second president/candidate you would like to compare:\t'))
    selections = [selection_index[int(selections[0])], selection_index[int(selections[1])]]
    num_words = int(input('What is the minimum number of words that should be included in each sample?\t'))
    print(f'\n\nSelection 1:\t\t{selections[0].title()}\nSelection 2:\t\t{selections[1].title()}\nVocabulary Size:\t{num_words}')
    return sorted(selections, reverse=True), num_words

selections, num_words = get_presidents()

## 2. Parse Text and Save Samples to Dataframe

In [ ]:
# Start timer to get total time for running all code
t0 = datetime.datetime.now()

# Save selections to variables
p1, p2 = sorted([selections[0], selections[1]])

# Create variable that identifies directory containing speech corpora
lvl1 = 'Corpus of Presidential Speeches/'

# Create dictionary to store samples. Will be converted to dataframe later.
speech_dict = {'president': [], 'name': [], 'text': []}

# Create dictionaries to keep track of labels and get speech count
labels_index = {}
speech_count = {}

# Iterate through each speech of selected presidents and save results to dictionaries created above
i=0
for folder in sorted(os.listdir(lvl1)):
    if folder in selections:
        labels_index[i] = folder.lower()
        speech_count[folder] = [0]
        
        for file in os.listdir(lvl1 + folder):
            speech_count[folder][0] += 1
            
            with open(f'{lvl1}{folder}/{file}', 'r') as f:
                data = f.read()
                
                # Remove tags for date, title, etc. and select punctuation
                no_tags = re.sub('<[^>]+>', '', data)
                no_line_breaks = no_tags.replace('\n', ' ')
                no_ellipses = no_line_breaks.replace('...', ' ')
                no_dashes = no_ellipses.replace('-', ' ')
                
                # Split data into sentences
                sentences = no_dashes.split('. ')
                
                # Create samples based on word count selection per sample
                # Sentences added to single sample until min word count is reached
                while len(sentences) > 0:
                    text = []
                    while (len(text) <= num_words) and (len(sentences) > 0):
                        sentence = sentences.pop(0)
                        
                        # Add period at end if ending character is not a question mark or exclamation point
                        if (len(sentence) > 0) and (sentence[-1] not in ['?', '!']):
                            sentence = sentence + '.'
                            
                        text += sentence.split()
                        
                    speech_dict['text'].append(' '.join(text))
                    speech_dict['president'].append(i)
                    speech_dict['name'].append(folder.title())
                    
        i+=1
                
df = pd.DataFrame.from_dict(speech_dict)
df['length'] = df['text'].apply(lambda x: len(x.split()))

# Final samples from each speech may not contain enough words. 
# Drop rows that do not contain minimum number of words
df = df[df['length'] >= num_words]

# Get number of classes in case comparing more than 2 in the future
num_classes = len(df['president'].unique())

print(df.shape)
print(labels_index)
df.head()

## 3. Data Cleaning
Text preprocessing was primarily adapted from the following blog post: [A Practitioner's Guide to Natual Language Processing](https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72).

Functions will be created for each step of preprocessing:
1. Expand Contractions
2. Remove Special Characters
3. Lemmatize Text
4. Remove Stop Words
5. Tokenize words

### Expand Cotractions

In [ ]:
from contractions import CONTRACTION_MAP

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    # Cxpand contractions into individual words using pre-defined list of contractions.
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        # Find words in text that match with contraction map, and return expanded text
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

### Remove Special Characters

In [ ]:
def remove_special_characters(text, remove_digits=False):
    # Remove special characters from text
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

### Lemmatize Text

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md', parse=True, tag=True, entity=True)

def lemmatize_text(text):
    # Get base form for word variants (i.e. running ==> run, cats ==> cat, etc.)
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

### Remove Stopwords & Tokenize

In [ ]:
import nltk
from nltk.tokenize.toktok import ToktokTokenizer

tokenizer = ToktokTokenizer()

# Remove words 'no' and 'not' from stopword list as these may change meaning of words/phrases
stopword_list = nltk.corpus.stopwords.words('english')
for word in ['no', 'not', 'he', 'she', 'his', 'her', 'hers']:
    stopword_list.remove(word)

# Add characters not captured in special character removal to stop list
stopword_list += ['[', ']', "\\"]                     

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

### Apply Preprocessing Functions to Data

In [ ]:
import unicodedata

clean_text = []

for text in df['text']:
    # Remove accent characters => Expand Contractions => Remove special characters => Lammatize Text => Remove Stop Words
    no_accent_chars = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    expanded_text = expand_contractions(no_accent_chars)
    no_special_chars = remove_special_characters(expanded_text, remove_digits=False)
    lem_text = lemmatize_text(no_special_chars)
    filtered_text = remove_stopwords(lem_text)
    
    # Make all text lowercase
    lowercase_text = filtered_text.lower()
    
    # Split text into list of words
    words = lowercase_text.split()
    clean_text.append(words)

# Add cleaned text to dataframe
df['clean_text'] = clean_text
df.head()

## 4. Explore Data
The primary concern with this data is that it may be imbalanced. If one individual gave significantly more speeches than the other, then our network may learn that it can maximize accuracy be simply guessing the more prominent speaker more frequently.

In [ ]:
plt.figure(figsize=(25, 8))
sns.countplot('name', data=df)
plt.title(f'Sample Count by {df.columns[0].title()}')
plt.ylabel('Sample Count')
plt.xlabel(df.columns[0].title())
plt.show()

### Balance Dataset
Because it is clear that one individual has significantly more samples than the other, it is best to select a subset of samples equivalent to the total sample size of the less prominent individual. This could later prove to be problematic for presidents who gave a limited number of public appearances, as this could result in there not being enough samples to generate meaningful results. In this case, it is not expected to be a problem.

In [ ]:
# Shuffle the Dataset.
shuffled_df = df.sample(frac=1)

# Create dictionary to store scaled dataframes for each president.
scaled_dfs = {}

# Set max samples based on president with fewest samples
n = shuffled_df.loc[shuffled_df['name'] == df['name'].value_counts().index[-1]].shape[0]

#Randomly select appropriate number of observations from the majority class
for name in df['name'].unique():
    scaled_dfs[f'{name}_df'] = shuffled_df.loc[shuffled_df['name'] == name].sample(n=n)

# Concatenate dataframes
scaled_dfs = [scaled_dfs[key] for key in list(scaled_dfs.keys())]
normalized_df = pd.concat(scaled_dfs)

#plot the dataset after the undersampling
plt.figure(figsize=(25, 8))
sns.countplot('name', data=normalized_df)
plt.title(f'Balanced Paragraph Count by {df.columns[0].title()}')
plt.ylabel('Paragraph Count')
plt.xlabel(df.columns[0].title())
plt.show()

In [ ]:
normalized_df.shape

In [ ]:
normalized_df.head()

In [ ]:
# Set variables for dependent and independent variables
target = normalized_df.president
data = normalized_df['clean_text'].values

### Create Bigrams
Certain terms may come up that make more sense together than independently (i.e. "United States", "North Korea", "Nuclear Weapon", etc). When terms are identified next to each other frequently, they may be combined into a single term using an underscore ("United_States"), making it easier for our network to deal with later on.

In [ ]:
from gensim.models import phrases

# Identify and create bigrams
bigrams = phrases.Phrases(data)
bigrams_data = []

for i in range(len(data)):
    bigrams_data.append(bigrams[data[i]])
    
bigrams_data = np.array(bigrams_data)

# Get word count for longest sample after stop words removed and bigrams created
max_len = 0
for bigram in bigrams_data:
    max_len = len(bigram) if len(bigram) > max_len else max_len
    
print(f'Maximum Sample Word Count: {max_len}\n')
print(f"Sample:\n{' '.join([bigram for bigram in bigrams_data[0]])}")

## 5. Vectorize Samples for Analysis Using GloVe
GloVe (short for Global Vectors Word Representation) is an open-source database of 6 billion pre-trained word vectors created by Stanford University. Rather than having to develop vector mapping for our entire vocabulary, theis database can be used to give our network a head start. Each unique word will be checked against the database to get only the vectors required for this specific dataset. Words and bigrams not found in the GloVe database will be assigned a zero vector and updated as part of the training process.

In [ ]:
# Create variable to hold set of all unique words in sample corpus
total_vocabulary = set(word for text in bigrams_data for word in text)
print(f'Total Words in Vocabulary: {len(total_vocabulary)}')

In [ ]:
# Create dictionary of vectors from GloVe based on total vocabulary
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    if word in total_vocabulary:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()

print(f'Total Words in Embedding Index: {len(embeddings_index)}')

In [ ]:
class W2vVectorizer(object):
    # Assign GloVe vector to each word in vocabulary, or zero vector to words/bigrams not found
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(embeddings_index))])
    
    # Following required to allow for implementation of a fit method
    # Cannot be used in SKLearn Pipeline otherwise 
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

## 6. Benchmarking
Traditional machine learning methods are much less resource intensive than neural networks, so it should be expected that a neural network will outperform them. Random forest, support vector classification, and logistic regression will be used as potential benchmarks, with the top performing model being used as the benchmark against which neural network performance will be compared.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf  = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(embeddings_index)),
                ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True, n_jobs=-1))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(embeddings_index)),
                ('Support Vector Machine', SVC())])
lr  = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(embeddings_index)),
                ('Logistic Regression', LogisticRegression(n_jobs=-1))])

In [ ]:
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

In [ ]:
# Use 2 cross-validations to gauge performance
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model in models]
scores

## 7. Creating Neural Networks
### Import Libraries

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, GRU
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D, Conv1D, MaxPooling1D, Flatten
from keras.models import Model, Sequential, load_model
from keras.metrics import categorical_crossentropy
from keras.preprocessing import text
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, RMSprop

### Keras Tokenization & Sequencing
For use in Keras, bigram data must be re-tokenized as sequences numerical indices rather than words. By rejoining each sample's list of bigrams into a string, the Keras tokenizer can handle this quickly and easily. The total words in the vocabulary should match the previous count for benchmarking.

In [ ]:
# Convert each sample's list of bigrams into a string
bigrams_joined = []

for bigrams in bigrams_data:
    bigrams_joined.append(' '.join(bigrams))

# Remove underscore from filters and no need to convert to lowercase
tokenizer = text.Tokenizer(filters='!"#%&()*+,./:;<=>?@[\\]^`{|}~\t\n', lower=False)
tokenizer.fit_on_texts(bigrams_joined)
sequences = tokenizer.texts_to_sequences(bigrams_joined)

word_index = tokenizer.word_index
print(f'Total Words in Vocabulary: {len(word_index)}')

### Padding
The neural network excpects each sample to be the same shape, however each sample is not the exact same length. A sequence of zeroes will be added to the end of each sample such that the length matches that of the longest sample in the corpus. 

Once padding and sequencing is completed, the original text and bigram data will be added as additional columns so that they can be retrieved by their respective indices later, allower for a more throrough analysis of the results later.

In [ ]:
X_t = pad_sequences(sequences, maxlen=max_len, padding='post')
X_t_df = pd.DataFrame(X_t)
X_t_df['bigrams'] = bigrams_data
X_t_df['text'] = normalized_df.reset_index(drop=True)['text']
X_t_df.head()

In [ ]:
# Set target variable
y = pd.get_dummies(target).values

### Train Test Split
Because the benchmark was calculated using 2-fold cross-validation (training and testing on half the data each time), the neural networks will be trained and tested in as similar a way as possible so as to maintain integrity of the results. Testing will be performed on half the data, while training will take place on the other half (validation split can be treated as a tunable hyperparameter).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_t_df, y, test_size=.5)

X_train_text = X_train[['text', 'bigrams']]
X_train = X_train[X_train.columns[:-2]]

X_test_text = X_test[['text', 'bigrams']]
X_test = X_test[X_test.columns[:-2]]

### Vectorization & Embedding
While the embedding index created earlier should remain valid (mapping each word to a vector using GloVe), it is performed again here with the Keras generated tokens just as a precaution. However, while creating the embedding matrix, there will be one difference in that words not found in the GloVe index will be assigned random vectors rather than empty vectors since they represent weights that will be tuned as part of the training process.

In [ ]:
# Create embedding index using GloVe
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
# Create matrix of random vectors which matches size of the word index
embedding_matrix = np.random.random((len(word_index) + 1, 100))
for word, i in word_index.items():
    
    # For each word found in the embedding index, assign the known vector
    embedding_vector = embeddings_index.get(word)
    
    # Otherwise, keep the randomly assigned vector
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1, 
                            100, weights = [embedding_matrix], 
                            input_length = max_len, trainable=True)

With the embedding matrix created, the search for a top performing neural network can begin.

### Network Development Strategy Synopsis
Convolutional Neural Networks (CNN) and Recurrent Neural Networks (RNN) have been shown to perform quite well for NLP classification tasks, and will therefore be used here in an attempt to identify the speakers of each sample. The two network architectures utilized here have been shown to perform well as a starting point for many NLP tasks, and allow for a significant amount of flexibility for adaptation. In addition to general architecture, the following strategies were implemented in building the network:

- **Grid Search:** Each network will be initialized with a function that utilizes the embedding layer already created, and will return the model, history, results, and processing time. All relevant hyperparameters (including number of epochs, validation split, etc.) are assigned default settings, and may be adjusted as desired by the user. A grid search will iterate through different combinations of hyperparameters searching for the minimum loss.

- **Loss vs Accuracy:** Through each epoch, the network will attempt to minimize the loss function. However, the best model will be selected based on its overall accuracy. This approach was selected in an effort to maintain alignment with Goodhart's Law, since accuracy is ultimately driven by the loss function.

- **Epochs & Early Stopping:** Each time the model is trained, it will be set to go through 100 epochs, but will stop if validation loss does not improve within 5 sequential epochs.

- **Checkpoints:** Because the total number of models generated can grow exponentially, it would be unrealistic to save each one to *.hdf5* for each set of hyperparameters. Instead, network accuracy and hyperparameters will be tracked, and the top performing model will be retrained so that predictions can be articulated. This is expected to result in small levels of variation within the results, but will nevertheless be reviewed at a later point to ensure the variations do not significantly impact the results. 


### Convolutional Network Architecture
The following function takes in a set of hyperparameters, and uses the existing embedding layer to create, train, and test a single convolutional neural network. It returns the model, model history, results, and total processing time.

In [ ]:
def create_cnn_model(cdim=128, ksize=5, pool1=5, pool2=35, density=128, lr=.001, epochs=100, batch_size=128, validation_split=.3, patience=5, verbose=0):
    
    # Start Timer
    start = datetime.datetime.now()
    
    
    # Display Hyperparameter Settings
    print('Convolution Dimensions\tWindow Size\tPool 1\t\tPool 2\t\tDensity')
    print(f'{cdim}\t\t\t{ksize}\t\t{pool1}\t\t{pool2}\t\t{density}')
    
    
    # Build Model
    input_ = Input(shape=(max_len,), dtype='int32')
    embedded_sequences = embedding_layer(input_)
    
    x = Conv1D(cdim, ksize, activation='relu')(embedded_sequences)
    x = MaxPooling1D(pool1, padding='same')(x)
    x = Conv1D(cdim, ksize, activation='relu')(x)
    x = MaxPooling1D(pool1, padding='same')(x)
    x = Conv1D(cdim, ksize, activation='relu')(x)
    x = MaxPooling1D(pool2, padding='same')(x)
    x = Flatten()(x)
    x = Dense(density, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_, outputs=x)
    
    
    # Compile Model
    history = model.compile(loss='categorical_crossentropy', 
                            optimizer=Adam(lr=lr), 
                            metrics=['accuracy'])
    
    
    # Create Checkpoints & Stopping Parameters
    checkpoints_path = f'cnn_best_{p1}_{p2}.hdf5'
    checkpoint = ModelCheckpoint(checkpoints_path, 
                                 monitor='val_loss', 
                                 verbose=verbose, 
                                 save_best_only=True, 
                                 mode='min')
    
    early_stopping = EarlyStopping(monitor='val_loss', 
                                   mode='min', 
                                   patience=patience)
    
    callbacks = [checkpoint, early_stopping]
    
    
    # Fit Model
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_split=validation_split, 
                        callbacks=callbacks, 
                        verbose=verbose)
    
    # Evaluate Model
    results = model.evaluate(X_test, y_test)
    
    # End Timer
    end = datetime.datetime.now()
    
    # Display Results
    print(f'Time to Complete:\t{end - start}')
    print(f'Loss:\t{results[0]:.2f}\tAccuracy:\t{results[1]:.4f}\n')
    
    return model, history, results, end-start

### CNN Grid Search
**Defining the Grid:** Due to limited resources, the grid was set up to create a wide range of hyperparameters, while simultaneously attempting to limit the total number of combinations.
One item worth noting is that, depending on the minimum word count selected, some pooling and window sizes will not work properly. These instances will simply be skipped.

In [ ]:
# Display total time elapsed since program started
t1 = datetime.datetime.now()
print(f'Time Elapsed: {t1 - t0}\n')

# Hyperparameter grid
cdims = [128, 256]
ksizes = [3, 5, 7]
pools1 = [3, 5, 7]
pools2 = [25, 35]
densities = [64, 128]

# Create dictionary for tracking CNN results and hyperparameters
cnn_results = {'Convolution Dimensions': [], 
               'Window Size': [], 
               'Pool 1': [], 
               'Pool 2': [], 
               'Density': [], 
               'Time': [], 
               'Loss': [], 
               'Accuracy': []}

for cdim in cdims:
    for ksize in ksizes:
        for pool1 in pools1:
            for pool2 in pools2:
                for density in densities:
                    
                    try:
                        cnn_model, cnn_history, cnn_eval, cnn_time = create_cnn_model(cdim=cdim, 
                                                                                      ksize=ksize, 
                                                                                      pool1=pool1, 
                                                                                      pool2=pool2, 
                                                                                      density=density)
                        cnn_results['Convolution Dimensions'].append(cdim)
                        cnn_results['Window Size'].append(ksize)
                        cnn_results['Pool 1'].append(pool1)
                        cnn_results['Pool 2'].append(pool2)
                        cnn_results['Density'].append(density)
                        cnn_results['Loss'].append(cnn_eval[0])
                        cnn_results['Accuracy'].append(cnn_eval[1])
                        cnn_results['Time'].append(cnn_time)
                        
                    except:
                        # If invalid hyperparameters, pass
                        pass
                    
t2 = datetime.datetime.now()
print(f'Time Elapsed: {t2 - t0}\n')
print(f'Time to Find Best Model: {t2 - t1}\n')

### Train Best CNN Model
Identify the model with the highest accuracy score, and use the associated hyperparameters to retrain the model, displaying a greater level of detail in verbosity when training.

In [ ]:
# Create dataframe from results dictionary
cnn_results_df = pd.DataFrame.from_dict(cnn_results)

# Get row values associated with highest accuracy
best_cnn_model = cnn_results_df[cnn_results_df['Accuracy'] == cnn_results_df['Accuracy'].max()]

# Save hyperparameters to variables
cdim = best_cnn_model['Convolution Dimensions'].values[0]
ksize = int(best_cnn_model['Window Size'].values[0])
pool1 = int(best_cnn_model['Pool 1'].values[0])
pool2 = int(best_cnn_model['Pool 2'].values[0])
density = int(best_cnn_model['Density'].values[0])

# Rebuild & retrain model with 
cnn_model, cnn_history, cnn_eval, cnn_time = create_cnn_model(cdim=cdim, 
                                                              ksize=ksize, 
                                                              pool1=pool1, 
                                                              pool2=pool2, 
                                                              density=density,
                                                              lr=.0001, 
                                                              epochs=100, 
                                                              patience=5, 
                                                              verbose=1)

t3 = datetime.datetime.now()
print(f'Time Elapsed: {t3 - t0}\n')
print(f'Time to Train Best Model: {t3 - t2}\n')

## Recurrent Network Architecture
The following function takes in a set of hyperparameters, and uses the existing embedding layer to create, train, and test a single convolutional neural network. It returns the model, model history, results, and total processing time.

**LSTM/GRU**: Unlike all other hyperparamters, the neuron type has an affect on the actual architecture of the network. The appropriate layer will be selected based on function input. While the default is set to *gru*, it should be noted that any other input will result in the model defaulting to *LSTM* and will not return an error.

**Bidirectional RNN**: Though more resource intensive, it was decided to use Bidirectional layers to improve accuracy.

In [ ]:
def create_rnn_model(rnn_type='gru', units=50, drop=.5, density=50, lr=.001, epochs=100, batch_size=128, validation_split=.3, patience=5, verbose=0):
    
    # Start Timer
    start = datetime.datetime.now()
    
    
    # Display Hyperparameter Settings
    model_type = 'GRU' if rnn_type == 'gru' else 'LSTM'
    print(f'Model Type:\t{model_type}\tUnits:\t{units}\tDropout Rate:\t{drop}\t\tDensity: {density}')
    
    
    # Build Model
    input_ = Input(shape=(max_len,), dtype='int32')
    embedded_sequences = embedding_layer(input_)
    
    if rnn_type == 'gru':
        x = Bidirectional(GRU(units, return_sequences=True))(embedded_sequences)
    else:
        x = Bidirectional(LSTM(units, return_sequences=True))(embedded_sequences)
        
    x = GlobalMaxPool1D()(x)
    x = Dropout(drop)(x)
    x = Dense(density, activation='relu')(x)
    x = Dropout(drop)(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_, outputs=x)
    
    
    # Compile Model
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(lr=lr), 
                  metrics=['accuracy'])
    
    
    # Create Checkpoints & Stopping Parameters
    checkpoints_path = f'rnn_best_{p1}_{p2}.hdf5'
    checkpoint = ModelCheckpoint(checkpoints_path, 
                                 monitor='val_loss', 
                                 verbose=verbose, 
                                 save_best_only=True, 
                                 mode='min')
    
    early_stopping = EarlyStopping(monitor='val_loss', 
                                   mode='min', 
                                   patience=patience)
    
    callbacks = [checkpoint, early_stopping]
    
    
    # Fit Model
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_split=validation_split, 
                        callbacks=callbacks, 
                        verbose=verbose)
    
    # Evaluate Model
    results = model.evaluate(X_test, y_test)
    
    # End Timer
    end = datetime.datetime.now()
    
    # Display Results
    print(f'Time to Complete:\t{end - start}')
    print(f'Loss:\t{results[0]:.2f}\tAccuracy:\t{results[1]:.4f}\n')
    
    return model, history, results, end-start

### RNN Grid Search
To account for the additional time required to train a Bidirectional RNN, fewer combinations of hyperparameters were used than were for the Convolutional Network.

In [ ]:
rnn_types = ['lstm', 'gru']
units = [25, 50]
drops = [.25, .5]
densities = [100, 150]

rnn_results = {'Model Type': [], 'Units': [], 'Dropout Rate': [], 'Density': [], 'Time': [], 'Loss': [], 'Accuracy': []}

for rnn_type in rnn_types:
    for unit in units:
        for drop in drops:
            for density in densities:
                rnn_model, rnn_history, rnn_eval, rnn_time = create_rnn_model(rnn_type=rnn_type, 
                                                                              units=unit, 
                                                                              drop=drop, 
                                                                              density=density)
                rnn_results['Model Type'].append(rnn_type)
                rnn_results['Units'].append(unit)
                rnn_results['Dropout Rate'].append(drop)
                rnn_results['Density'].append(density)
                rnn_results['Loss'].append(rnn_eval[0])
                rnn_results['Accuracy'].append(rnn_eval[1])
                rnn_results['Time'].append(rnn_time)
                
t4 = datetime.datetime.now()
print(f'Time Elapsed: {t4 - t0}\n')
print(f'Time to Find Best Model: {t4 - t3}\n')

### Train Best RNN Model
Identify the model with the highest accuracy score, and use the associated hyperparameters to retrain the model, displaying a greater level of detail in verbosity when training.

In [ ]:
rnn_results_df = pd.DataFrame.from_dict(rnn_results)

best_rnn_model = rnn_results_df[rnn_results_df['Loss'] == rnn_results_df['Loss'].min()]

rnn_type = best_rnn_model['Model Type'].values[0]
units = int(best_rnn_model['Units'].values[0])
drop = float(best_rnn_model['Dropout Rate'].values[0])
density = int(best_rnn_model['Density'].values[0])

rnn_model, rnn_history, rnn_eval, rnn_time = create_rnn_model(rnn_type=rnn_type, 
                                                              units=units, 
                                                              drop=drop, 
                                                              density=density, 
                                                              lr=.001, 
                                                              epochs=100, 
                                                              patience=5, 
                                                              verbose=1)

t5 = datetime.datetime.now()
print(f'Time Elapsed: {t5 - t0}\n')
print(f'Time to Train Best Model: {t5 - t4}\n')

### Visualize Results

In [ ]:
from operator import itemgetter

labels = [f'Benchmark: {max(scores,key=itemgetter(1))[0]}', 'CNN', 'RNN']
acc = [max(scores,key=itemgetter(1))[1], cnn_eval[1], rnn_eval[1]]

plt.figure(figsize = (15, 10))
plt.bar(labels, acc)
for i in range(3):
    plt.text(labels[i], acc[i] + .02, f'{np.round(acc[i]*100,2)}%', ha='center')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1)
plt.yticks([i/10 for i in range(5, 11)], [f'{i*10}%' for i in range(5, 11)])
plt.title('Top Scores')
plt.show()


### Inspect Top Performing Model
#### Model Summary

In [ ]:
top_models = {'rnn': rnn_model, 'cnn': cnn_model}
histories = {'rnn_hist': rnn_history, 'cnn_hist': cnn_history}
best = 'c' if cnn_eval[1] > rnn_eval[1] else 'r'
best_model = top_models[f'{best}nn']
best_model_hist = histories[f'{best}nn_hist']
print(f'Top Model Type: {best.upper()}NN')
best_model.summary()

#### Results History

In [ ]:
# Display history for accuracy
plt.figure(figsize=(15, 10))
plt.plot(best_model_hist.history['acc'])
plt.plot(best_model_hist.history['val_acc'])
plt.title(f'{best.upper()}NN Model Accuracy')
plt.ylabel(f'{best.upper()}NN Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Display history for loss
plt.figure(figsize=(15, 10))
plt.plot(best_model_hist.history['loss'])
plt.plot(best_model_hist.history['val_loss'])
plt.title(f'{best.upper()}NN Model Loss / Error')
plt.ylabel(f'{best.upper()}NN Loss / Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


### Predictions
Create a confusion matrix to inspect the number of accurate and inaccurate predictions.

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
preds = np.argmax(best_model.predict(X_test), axis=1)
actual = np.argmax(y_test, axis=1)

cm = confusion_matrix(actual, preds)
cm_plot_labels = [p1.title(), p2.title()]

plot_confusion_matrix(cm, cm_plot_labels, normalize=False, title=f'{best.title()}NN Confusion Matrix')

print(f'F1 Score: {f1_score(actual, preds)*100:.2f}%')

Out of 1,780 predictions, the network made 123 errors. A slight majority of the errors were predictions that statements were made by Trump, when they were in fact made by Clinton. However, for this project, the direction of the error is of little consequence. Therefore, the F1-Score is being used as the primary metric.

## View List of Inaccurate Predictions

In [ ]:
# Save all predictions to dataframe
preds_df = pd.DataFrame.from_dict({'Predictions': preds, 'Actual': actual})
preds_df = preds_df.join(X_test_text.reset_index(drop=True))

# Create column to indicate if the prediction matches the actual
# 0 = Clinton; 1 = Trump
preds_df['Accuracy'] = preds_df['Predictions'] == preds_df['Actual']
preds_df.head()

In [ ]:
# Create dictionary for sorting predictions
word_counts = {f'{p1.title()}_Correct':[], 
               f'{p1.title()}_Incorrect':[], 
               f'{p2.title()}_Correct':[], 
               f'{p2.title()}_Incorrect':[]}

error_tracking = []

# Assign each text sample to dictionary
# Print actual speaker and text for each inaccurate prediction
for i in preds_df.index:
    if preds_df.iloc[i]['Actual'] == 0:
        if preds_df.iloc[i]['Accuracy'] == True:
            word_counts[f'{p1.title()}_Correct'] += preds_df.iloc[i]['bigrams']
        else:
            word_counts[f'{p1.title()}_Incorrect'] += preds_df.iloc[i]['bigrams']
            error_tracking += preds_df.iloc[i]['bigrams']
            print(f'{p1.title()}:')
            print(preds_df.iloc[i]['text'], '\n')
    else:
        if preds_df.iloc[i]['Accuracy'] == True:
            word_counts[f'{p2.title()}_Correct'] += preds_df.iloc[i]['bigrams']
        else:
            word_counts[f'{p2.title()}_Incorrect'] += preds_df.iloc[i]['bigrams']
            error_tracking += preds_df.iloc[i]['bigrams']
            print(f'{p2.title()}:')
            print(preds_df.iloc[i]['text'], '\n')

### Sample Observations: Do errors highlight similarities?
Share your thoughts here.

### Visualizing the Differences with Word Clouds
The first step is to count the frequency with which each word is used by each candidate.

In [ ]:
word_counts = {}

for key in word_index.keys():
    word_counts[key] = {p1: 0, p2: 0}
    
for i in range(preds_df.shape[0]):
    for word in preds_df.iloc[i]['bigrams']:
        selection = p1 if preds_df.iloc[i]['Actual'] == 0 else p2
        try:
            word_counts[word.lower()][selection]+=1
        except:
            pass
        
for key in list(word_counts.keys())[:5]:
    print(f'{key}: {word_counts[key]}')

#### Calculate Word Usage Disparity
Rather than simply highlighting the most frequently used words by each candidate, it is more desireable to look at the words with the greatest disparity in ussage between the candidates on a percentage basis. The top 500 words with the greatest percentage disparity will be used.

*Note: when a word is used frequently by one candidate but never used by the other, 0.5 is subsituted for 0 when calculating the percentage to avoid null/undefined values.*

In [ ]:
word_counts_df = pd.DataFrame.from_dict(word_counts).T
word_counts_df['difference'] = word_counts_df[p1] - word_counts_df[p2]
# word_counts_df['abs_difference'] = np.abs(word_counts_df['difference'])

# Replace 0 values with 0.5
word_counts_df = word_counts_df.replace(to_replace=0, value=.5)

word_counts_df[f'{p1}_portion'] = word_counts_df[p2] / word_counts_df[p1]
word_counts_df[f'{p2}_portion'] = word_counts_df[p1] / word_counts_df[p2]
word_counts_df['pct_diff'] = word_counts_df[[f'{p2}_portion', f'{p1}_portion']].max(axis=1)

# Reset 0.5 values to 0
word_counts_df = word_counts_df.replace(to_replace=.5, value=0)

word_cloud_df = word_counts_df.sort_values(by='pct_diff', ascending=False)[:1000]
word_cloud_df.head()

### Convert Word Counts to Word List String

In [ ]:
from wordcloud import WordCloud, STOPWORDS
from IPython.display import Image as im
from PIL import Image

# Create list of words based on counts in dataframe
p1_cloud = []
p2_cloud = []

p1_word_cloud_df = word_cloud_df[word_cloud_df['difference'] > 0]
p2_word_cloud_df = word_cloud_df[word_cloud_df['difference'] < 0]

for i, w in enumerate(list(p1_word_cloud_df.index)):
    for j in range(int(list(p1_word_cloud_df[p1])[int(i)])):
        p1_cloud.append(w)
        
for i, w in enumerate(list(p2_word_cloud_df.index)):        
    for k in range(int(list(p2_word_cloud_df[p2])[int(i)])):
        p2_cloud.append(w)

# Words repeated in sequence cause them 
# to be duplicated in word cloud
# Shuffle lists to solve problem
np.random.shuffle(p1_cloud)
np.random.shuffle(p2_cloud)

# Convert lists to strings
p1_cloud = ', '.join(p1_cloud)
p2_cloud = ', '.join(p2_cloud)

# Create cloud for errors
error_cloud = []
for word in error_tracking:
    if len(word)>10: # in word_cloud_df.index[:500]:
        error_cloud.append(word)
error_cloud = ', '.join(error_cloud)  

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
colors = ["#FF0000", "#FF6347", "#DC143C", "#0000FF", "#0000CD", "#4169E1"]
cmap = LinearSegmentedColormap.from_list("mycmap", colors)

# No shape mask
p1_wc = WordCloud(background_color='white', colormap='Blues', max_words=500, width=1000, height=500)
p1_wc.generate(p1_cloud)

p2_wc = WordCloud(background_color='white', colormap='Reds', max_words=500, width=1000, height=500)
p2_wc.generate(p2_cloud)

error_wc = WordCloud(background_color='white', colormap=cmap, max_words=500, width=1000, height=500)
error_wc.generate(error_cloud)

# Display word clouds
plt.figure(figsize=(20, 35))
plt.imshow(p1_wc)
plt.axis('off')
plt.title(p1.title())
plt.show()

plt.figure(figsize=(20, 35))
plt.imshow(p2_wc)
plt.axis('off')
plt.title(p2.title())
plt.show()

plt.figure(figsize=(20, 35))
plt.imshow(error_wc)
plt.axis('off')
plt.title('Themes in Errors')
plt.show()

In [ ]:
T1 = datetime.datetime.now()
print(f'Time to Complete: {T1 - t0}\n')

### Observations
Document your own observations here.

## Conclusions & Next Steps
What are your conclusions?